In [1]:
# Thanks to Kenarapfaik and Kej Jee for Guidance 

from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [2]:
def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
#         try:
#             driver.find_element_by_class_name("selected").click()
#         except ElementClickInterceptedException:
#             pass

        time.sleep(.1)
        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("react-job-listing")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            try:
                driver.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
#             print(' x out worked')
            except NoSuchElementException:
#             print(' x out failed')
                pass

            collected_successfully = False
            
            while not collected_successfully:
                try:
                    print('Collecting')
                    rating = driver.find_element_by_xpath('.//span[@class="css-1m5m32b e1tk4kwz2"]').text
                    company_name = driver.find_element_by_xpath('.//div[@class="css-87uc0g e1tk4kwz1"]').text
                    company_name = company_name.replace("\n"+rating, "")
                    location = driver.find_element_by_xpath('.//div[@class="css-56kyx5 e1tk4kwz5"]').text
                    job_title = driver.find_element_by_xpath('.//div[@class="css-1vg6q84 e1tk4kwz4"]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text            
                    collected_successfully = True
                except:
                    company_name = -1
                    rating = -1
                    location = -1
                    job_title = -1
                    job_description = -1
                    size = -1
                    collected_successfully = True

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="css-56kyx5 css-16kxj2j e1wijj242"]').text
            
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))     

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location            
                        })
            #add job to jobs
            
            
        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.


In [3]:
path = "/usr/bin/chromedriver"
df = get_jobs("data scientists", 200, False, path, 5)
df.to_csv('Glassdoor_jobs.csv', index= False)
print('Created the CSV file for Glassdoor Jobs.')

Progress: 0/200
Collecting
Progress: 1/200
Collecting
Progress: 2/200
Collecting
Progress: 3/200
Collecting
Progress: 4/200
Collecting
Progress: 5/200
Collecting
Progress: 6/200
Collecting
Progress: 7/200
Collecting
Progress: 8/200
Collecting
Progress: 9/200
Collecting
Progress: 10/200
Collecting
Progress: 11/200
Collecting
Progress: 12/200
Collecting
Progress: 13/200
Collecting
Progress: 14/200
Collecting
Progress: 15/200
Collecting
Progress: 16/200
Collecting
Progress: 17/200
Collecting
Progress: 18/200
Collecting
Progress: 19/200
Collecting
Progress: 20/200
Collecting
Progress: 21/200
Collecting
Progress: 22/200
Collecting
Progress: 23/200
Collecting
Progress: 24/200
Collecting
Progress: 25/200
Collecting
Progress: 26/200
Collecting
Progress: 27/200
Collecting
Progress: 28/200
Collecting
Progress: 29/200
Collecting
Scraping terminated before reaching target number of jobs. Needed 200, got 30.
Created the CSV file for Glassdoor Jobs.
